## 전체 이미지 데이터에 대하여 선명도 전처리 작업실행

In [2]:
import cv2
import os
import numpy as np
import glob
from skimage.exposure import match_histograms
from tqdm import tqdm  # 진행상황 표시바

def process_batch_images(input_folder, output_folder, reference_image_path):
    # 1. 폴더 생성 (없으면 생성)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"📁 결과 저장 폴더 생성 완료: {output_folder}")

    # 2. 기준 이미지(Reference) 로드
    # 의료 영상(X-ray)은 흑백(Grayscale)으로 처리하는 것이 일반적입니다.
    ref_img = cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE)
    
    if ref_img is None:
        print("❌ 기준 이미지를 찾을 수 없습니다. 경로를 확인해주세요.")
        return

    # 3. CLAHE 객체 생성 (미리 만들어두고 재사용)
    # clipLimit: 대비 제한 값 (높을수록 더 강하게 대비를 줌, 보통 2.0~4.0 사용)
    # tileGridSize: 이미지를 나누는 그리드 크기 (8x8이 표준)-
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    # 4. 대상 이미지 파일 목록 가져오기 (png, jpg, jpeg 등)
    # 확장자는 필요에 따라 수정하세요.
    image_files = glob.glob(os.path.join(input_folder, '*.[jp][pn]g')) # jpg, png 모두 잡음
    image_files.extend(glob.glob(os.path.join(input_folder, '*.jpeg')))

    print(f"🚀 총 {len(image_files)}장의 이미지 처리를 시작합니다...")

# 5. 반복 처리
    for file_path in tqdm(image_files):
        try:
            # (1) 이미지 읽기
            filename = os.path.basename(file_path)
            src_img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            
            if src_img is None:
                continue

            # (2) 히스토그램 매칭 (Source -> Reference 분포 복사)
            # scikit-image는 결과를 float64로 반환하므로 uint8로 변환 필수
            # 사용 목적 : ref이미지와 대비도의 영역범위를 맞추기 위함. 
            matched = match_histograms(src_img, ref_img, channel_axis=None)
            matched = matched.astype('uint8')

            # (3) CLAHE 적용 (선명도 개선)
            final_result = clahe.apply(matched)

            # (4) 결과 저장
            save_path = os.path.join(output_folder, filename)
            cv2.imwrite(save_path, final_result)

        except Exception as e:
            print(f"⚠️ 에러 발생 ({filename}): {e}")

    print("✅ 모든 처리가 완료되었습니다.")



In [ ]:
# --- 실행 설정 ---
if __name__ == "__main__":
    # 여기에 실제 경로를 입력하세요
    INPUT_DIR = "C:/project/lungX/CXR8/image_set/PA/No Finding"        # 원본 이미지들이 있는 폴더
    OUTPUT_DIR = "C:/project/lungX/CXR8/image_set/PA/No Finding/processed_xrays" # 결과물이 저장될 폴더
    
    
    REF_IMG = "C:/project/lungX/CXR8/image_set/AP/No_Finding/00004021_001.png"      # 기준이 될 '가장 잘 나온' 이미지 1장


    process_batch_images(INPUT_DIR, OUTPUT_DIR, REF_IMG)

In [6]:
# --- 실행 설정 ---
# 성능검증하려고 사용하는 데이터. JSRT
if __name__ == "__main__":
    # 여기에 실제 경로를 입력하세요
    INPUT_DIR = "D:/nodules-in-chest-xrays-jsrt/images"        # 원본 이미지들이 있는 폴더
    OUTPUT_DIR = "D:/nodules-in-chest-xrays-jsrt/images_clean" # 결과물이 저장될 폴더
    
    
    REF_IMG = "D:/lung_xray/images/00000010_000.png"    # 기준이 될 '가장 잘 나온' 이미지 1장


    process_batch_images(INPUT_DIR, OUTPUT_DIR, REF_IMG)

🚀 총 247장의 이미지 처리를 시작합니다...


100%|██████████| 247/247 [00:36<00:00,  6.79it/s]

✅ 모든 처리가 완료되었습니다.
